In [1]:
import xarray as xr
import rydanalysis as ra
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm
from scipy.stats import sem

%matplotlib widget

In [2]:
path = Path.cwd().parent / "raw_data"

live_analysis_path = Path.cwd().parent / "Analysis" 
peaks_file = live_analysis_path / "peaks.h5"
path

WindowsPath('w:/Projekte - Projects/2020_Aging/2021_03_11/05_nWHH60_with_calibration/raw_data')

In [3]:
data = ra.load_data(path, to_multiindex=True, lazy=None)
variable = data.ryd_data.variables.columns[0]
variable

2021-03-11 16:52:08.712 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


'nWAHUHA'

In [4]:
peak_df = data.scope_traces[3:-1].peaks_summary.get_peak_description(width=2e-9, height=0.0009, distance=None)
peak_df.to_csv("peak_df.csv")

In [6]:
summary = ra.summarize_peak_description(peak_df)
counts = summary.counts
counts_mean = counts.groupby(variable).mean()
counts_sem = counts.groupby(variable).reduce(sem)

In [13]:
summary_n_tot = summary.sel(InitON=1, nWAHUHA=0, DPON=0, drop=True)
summary_de = summary.sel(InitON=0, nWAHUHA=0, DPON=1, drop=True)
summary_z = summary.sel(InitON=0, DPON=1, drop=True)

In [32]:
def plot_counts(summary, ax):
    counts = summary.counts
    counts_mean = counts.groupby(variable).mean()
    counts_sem = counts.groupby(variable).reduce(sem)
    ax.errorbar(counts_mean[variable], counts_mean, counts_sem, linestyle='', marker='o')

def plot_calib(summary, x, ax, label="", color="tab:blue"):
    counts = summary.counts
    counts_mean = float(counts.mean())
    counts_sem = float(counts.reduce(sem))

    ones = np.ones(len(x))
    ax.plot(
        x,
        counts_mean*ones,
        c=color,
        label=label,
    )
    ax.fill_between(
        x,
        (counts_mean + counts_sem) * ones,
        (counts_mean - counts_sem) * ones,
        color=color,
        alpha=0.5
    )

In [37]:
variable

'nWAHUHA'

In [38]:
x = np.unique(data[variable].values)

fig, ax = plt.subplots()

plot_counts(summary_z, ax)
plot_calib(summary_n_tot, x, ax, "n_tot", "tab:green")
plot_calib(summary_de, x, ax, "de", "tab:red")
ax.legend()
ax.set_xlabel(variable)
ax.set_ylabel("counts")

fig.tight_layout()
fig.savefig("plot_counts.png", dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close("all")